In [1]:
""" Auto-number FRS items from formatted list
*+FRS1234+*
#
##
###
*** Non-numbered FRS text or sub-item
#
##
###

becomes 

*+FRS1234+*
* *FRS1234.1:*
** *FRS1234.1.1:*
*** *FRS1234.1.1.1:*
**** Non-numbered FRS text or sub-item

* *FRS1234.2:*
** *FRS1234.2.1:*
*** *FRS1234.2.1.1:* 
"""


' Auto-number FRS items from formatted list\n*+FRS1234+*\n#\n##\n###\n*** Non-numbered FRS text or sub-item\n#\n##\n###\n\nbecomes \n\n*+FRS1234+*\n* *FRS1234.1:*\n** *FRS1234.1.1:*\n*** *FRS1234.1.1.1:*\n**** Non-numbered FRS text or sub-item\n\n* *FRS1234.2:*\n** *FRS1234.2.1:*\n*** *FRS1234.2.1.1:* \n'

In [5]:
import re
subitem_match = re.compile(r"^(?:([#]+)\s*)").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match
header_match = re.compile(r"^(?:([\*\#]*) )?([\+\*]+)([^\*\+]+?)\:?([\+\*]+)\:?\s*(.+)?$").match
frs_match = re.compile(r"([FU]RS\d*)\.?([\d\.]+)?").match
num_escape_match = re.compile(r"^(\*{2})(\#+)").match
image_caption_match = re.compile(r"(Image \d+)").match
toplevel_match = re.compile(r"^([\+\*]+)([^\*\+]+?)\:?([\+\*]+)\:?\s*(.+)?$").match

class FRSNumberStateMachine():
    VALID_INDENTS = "*+"
    
    # inputs
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    LINE_HEADER = 3
    LINE_NUM_ESCAPE = 4
    LINE_IMG_CAPTION = 5
    
    # states
    ST_INIT = 0
    ST_NONUM = 1
    ST_FRS = 2
    
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = []
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
        self.sm_st = self.ST_INIT
        self.sm = [
            # ST_INIT             ST_NONUM             ST_FRS
            [self.set_frs,        self.set_frs,        self.set_frs],        # LINE_FRS
            [None,                None,                self.set_subitem],    # LINE_SUBITEM
            [None,                None,                None],                # LINE_OTHER
            [self.set_header,     self.set_header,     self.set_header],     # LINE_HEADER
            [self.set_num_escape, self.set_num_escape, self.set_num_escape], # LINE_NUM_ESCAPE
            [None,                None,                None],                # LINE_IMG_CAPTION
        ]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
        
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
    
    def format_lvl(self, lvl):
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def feed(self, line):
        ev, arg = self._parse_input(line)
        func = self.sm[ev][self.sm_st]
        if func:
            st, rv = func(arg)
            self.sm_st = st
        else:
            rv = line
        return rv
    
    def _parse_input(self, line): 
        """Parse the input line to determine the 
        type of input, and any arguments to be passed
        to the input event handler.
        """
        m = header_match(line)
        if m:
            ind, fmt, s, fmt2, _ = m.groups()

            # Verify format to avoid false positive
            # if other * or + characters appear in the string
            if fmt == fmt2[::-1]:
                m2 = frs_match(s)
                if m2:
                    typ = self.LINE_FRS
                    frs, nums = m2.groups()
                    arg = (line, ind, fmt, frs, nums)
                    return typ, arg
                if image_caption_match(s):
                    return self.LINE_IMG_CAPTION, ()
                else:
                    typ = self.LINE_HEADER
                    arg = (line, ind, fmt, s)
                return typ, arg
        m = subitem_match(line)
        if m:
            typ = self.LINE_SUBITEM
            arg = line, len(m.group(1) or "")
            return typ, arg
        m = num_escape_match(line)
        if m:
            typ = self.LINE_NUM_ESCAPE
            arg = line,
            return typ, arg
        return self.LINE_OTHER, (line,)
    
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def parse_text(self, text):
        for line in text.splitlines():
            yield self.feed(line)
    
    # State machine methods
                                
    def set_frs(self, arg):
        line, indent, fmt, frs, nums = arg
        self.ilvl = len(indent or "")
        self.fmt = fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        return self.ST_FRS, line
    
    def set_header(self, arg):
        return self.ST_NONUM, arg[0]
    
    def set_subitem(self, arg):
        line, n = arg
        self.set_indent(n)
        fmt = self.frs_string()
        try:
            _, tail = line.split(" ", 1)
        except ValueError:
            v = fmt
        else:
            v = " ".join((fmt, tail))
        return self.sm_st, v
    
    def set_num_escape(self, arg):
        line, = arg
        return self.sm_st, line.lstrip("*")
    
NumState = FRSNumberStateMachine

In [6]:
ntests = 0
passed = 0
errors = []
def assert_ab(a, b, equal=True):
    global ntests, passed, errors
    ntests += 1
    try:
        if equal:
            assert a == b, "%r != %r" % (a, b)
        else:
            assert a != b, "%r == %r" % (a, b)
    except AssertionError as e:
        val = str(e)
        if isinstance(a, str) and isinstance(b, str):
            val += "\n" + str(a.splitlines()) + "\n" + str(b.splitlines())
        errors.append(val)
    else:
        passed += 1

def assert_header_match(s, exp):
    assert_ab(header_match(s).groups(), exp)
        
assert header_match("**") == None
assert header_match("***") == None
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("*URS*", (None, "*", "URS", "*", None))
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*", ("*", "*", "FRS1234", "*", None))
assert_header_match("*FRS1234.5.6*", (None, "*", "FRS1234.5.6", "*", None))
assert_header_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234.5.6", "+*", None))
assert_header_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234.5.6.7.8", "+*", None))
assert_header_match("* *FRS1234:*", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*:", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", "*", "Bob"))



def assert_num_escape(s, exp):
    m = num_escape_match(s)
    r = m.groups() if m else None
    assert_ab(r, exp)
    
assert_num_escape("**# Bob", ("**", "#"))
assert_num_escape("**## Bob", ("**", "##"))
assert_num_escape("**### Bob", ("**", "###"))
assert_num_escape("# Bob", None)
assert_num_escape("## Bob", None)
assert_num_escape("### Bob", None)
assert_num_escape("* Bob", None)
assert_num_escape("** Bob", None)
assert_num_escape("*** Bob", None)

#def assert_img_match(s):
#    m = image_match(s)
#    assert_ab(bool(m), True)
    
#assert_img_match("Image 1")

def set_str(st, s):
    ind, fmt, fs, _, _ = header_match(s).groups()
    frs, nums = frs_match(fs).groups()
    st.set_frs((s, ind, fmt, frs, nums))
    

def test_state(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1*", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1*", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    set_str(st, s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    set_str(st, s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

def assert_feed_indent(s, feed, exp):
    st = NumState()
    set_str(st, s)
    st.sm_st = st.ST_FRS
    st.feed(feed)
    assert_ab(st.frs_string(), exp)
    
assert_feed_indent("*FRS1234*", "# Bob", "* *FRS1234.1:*")
assert_feed_indent("*FRS1234*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "# Bob", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## Bob", "** *FRS1234.1.1.1:*")
assert_feed_indent("* *FRS1234.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("* *FRS1234*", "## Bob", "** *FRS1234.1:*")
assert_feed_indent("** *FRS1234.1*", "## Bob", "** *FRS1234.2:*")
assert_feed_indent("* *FRS1234.1*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("** *FRS1234.1.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("*FRS1234.1*", "# ", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## ", "** *FRS1234.1.1.1:*")
assert_feed_indent("*FRS1234.1*", "#", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "##", "** *FRS1234.1.1.1:*")

def assert_double_parse(t, d=5):
    r = list(NumState(d).parse_text(t))
    r2 = list(NumState(d).parse_text("\n".join(r)))
    assert_ab(r, r2)
    
dpt = """*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""
assert_double_parse(dpt)

def assert_new_state(s, exp, nxt=1, ind=0):
    st = NumState()
    for l in s.splitlines():
        st.feed(l)
    for _ in range(nxt):
        st.next()
    for _ in range(ind):
        st.indent()
    assert_ab(st.frs_string(), exp)
    
assert_new_state("*FRS123.1*\n*FRS31.2*", "*FRS31.3:*")
assert_new_state("*FRS123.1*\n*FRS123.3*", "*FRS123.4:*")

def assert_parse_text(txt, exp):
    st = NumState()
    lines = list(st.parse_text(txt))
    res = "\n".join(lines)
    assert_ab(res, exp)

lines = """*FRS1234.1:*
#"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*"""
assert_parse_text(lines, exp)

lines="""*FRS1234.1:*
#
## Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
#
##
"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*
** *FRS1234.1.1.1:* Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
* *FRS1234.2.1:*
** *FRS1234.2.1.1:*"""

assert_parse_text(lines, exp)

lines = "+*FRS*+\n*FRS1234:*\n#"
exp = "+*FRS*+\n*FRS1234:*\n* *FRS1234.1:*"
assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
*** Bob
## Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
*** Bob
** *FRS2908.1.1.1:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
#### Biff
*** Bob
# Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
**** *FRS2908.1.1.1.1.1:* Biff
*** Bob
* *FRS2908.1.2:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.0*
# Bob1
## Bob2
*** Fred1
*** Fred2
## Bob3
## Bob4
## Bob5
*** Fred3
#### Bob6
#### Bob7
#### Bob8
*** Fred4
# Bob9
"""

exp = """*FRS2908.0*
* *FRS2908.0.1:* Bob1
** *FRS2908.0.1.1:* Bob2
*** Fred1
*** Fred2
** *FRS2908.0.1.2:* Bob3
** *FRS2908.0.1.3:* Bob4
** *FRS2908.0.1.4:* Bob5
*** Fred3
**** *FRS2908.0.1.4.1.1:* Bob6
**** *FRS2908.0.1.4.1.2:* Bob7
**** *FRS2908.0.1.4.1.3:* Bob8
*** Fred4
* *FRS2908.0.2:* Bob9"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** Fred:
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** Fred:
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** considered +/- 15 degrees
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** considered +/- 15 degrees
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*FRS123* 
# Bob
*Image 1*: Bob's image
# Bob"""

exp = """*FRS123* 
* *FRS123.1:* Bob
*Image 1*: Bob's image
* *FRS123.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*URS2908.0* 
# Bob
** considered +/- 15 degrees
# Bob"""

exp = """*URS2908.0* 
* *URS2908.0.1:* Bob
** considered +/- 15 degrees
* *URS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

def assert_parse_empty(s, exp):
    st = NumState()
    res = None
    for res in st.parse_text(s):
        pass
    assert_ab(res, exp)
    
assert_parse_empty("*User Requirements*\n# Bar", "# Bar")

def finish():
    print("%d/%d tests passed successfully" % (passed, ntests))
    for e in errors:
        print(e)
    if errors:
        raise Exception("Unit testing suite failed")
finish()

95/95 tests passed successfully


In [7]:
# toplevel_match = re.compile(r"^([\+\*]+)([^\*\+]+?)\:?([\+\*]+)\:?\s*(.+)?$").match
# def assert_ab2(a, b, equal=True):
#     if equal:
#         assert a == b, "%r != %r" % (a, b)
#     else:
#         assert a != b, "%r == %r" % (a, b)
    
            
# def assert_toplevel_match(s, exp):
#     assert_ab(toplevel_match(s).groups(), exp)
# assert_ab2(toplevel_match("Test"), None)
# assert_ab2(toplevel_match("* *+Test Test+*"), None)

In [12]:
import clipboard

In [27]:
template=  r'''lines=r"""%s
"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
#clipboard.copy(res)
'''
s = template % clipboard.paste()
clipboard.copy(s)
exec(s)

*+URS3653.1+* The software will be able to control the pH

*+FRS3653.1+* The pH controller will have 2 outputs: % CO2 composition in Main Gas, and % Base Pump Duty.
*+FRS3653.2+* Auto mode - software takes the user set point plus the pH "Deadband" setting, and varies the CO2 output to achieve it, based on the present value reported by the sensor and the CO2 PID settings. It also takes the user set point minus the pH "Deadband" setting, and varies the Base output to achieve it, based on the present value reported by the sensor and the Base PID settings.
*+FRS3653.3+* Manual mode - CO2 and Base output are what the user requested.
*+FRS3653.4+* Off mode - CO2 and Base output are 0.
-*+FRS3653.5+*- Broken Sensor mode - when the user has requested Auto mode but the pH PV reported by the sensor is either above the pH "Valid High" setting or below the pH "Valid Low" setting, the software outputs the average of the last 100 seconds of CO2 and Base output.
*+FRS3653.6+* The pH PV reported by a 

In [32]:
lines=r"""*+URS3653.1+* The software will be able to control the pH

*+FRS3653+* pH Control
* Note: All system variables in the following items will be located under the category "Temperature" unless specified otherwise.
*+FRS3653.1+* The pH controller will have 2 outputs: % CO2 composition in Main Gas, and % Base Pump Duty.
-*+FRS3653.2+*- Auto mode - software takes the user set point plus the pH "Deadband" setting, and varies the CO2 output to achieve it, based on the present value reported by the sensor and the CO2 PID settings. It also takes the user set point minus the pH "Deadband" setting, and varies the Base output to achieve it, based on the present value reported by the sensor and the Base PID settings.
-*+FRS3653.3+*- Manual Mode - CO2 and Base output are what the user requested.
*+FRS3653.4+* Off mode - CO2 and Base output are 0.
-*+FRS3653.5+*- Broken Sensor mode - when the user has requested Auto mode but the pH PV reported by the sensor is either above the pH "Valid High" setting or below the pH "Valid Low" setting, the software outputs the average of the last 100 seconds of CO2 and Base output.
*+FRS3653.6+* The pH PV reported by a sensor is considered invalid if:
* *+FRS3653.6.1+* It is either above the pH "Valid High" setting or below the pH "Valid Low" setting
* *+FRS3653.6.2+* It has changed by the pH "Rate Fail Delta PV" setting within the pH "Rate Fail Delta Time (s)" setting time.
*+FRS3653.7+* Broken Sensor mode - when the user has requested Auto mode but the pH PV is invalid (see above), the software outputs the average of the last 100 seconds of CO2 and Base output.
# The controller will be configurable to allow the use of one or two pH sensors as pH A and pH B. 
# HMI 
## Users will be able to select one of three modes: Auto, Manual, or Off.
## The current PV, SP, and mode will be displayed to the user with appropriate units.
## Interlock or broken status will be displayed to the user.
# pH Detection
## pH will be detected by a hardware sensor. 
## A linear calibration (y = mx + b) will be used to translate the hardware signal into the Process Value (PV).
## System Variable "A Use Temp Comp?" will be used to determine whether to correct pH A PV for temperature using the Nernst equation.
## System Variable "B Use Temp Comp?" will be used to determine whether to correct pH B PV for temperature using the Nernst equation.
# Auto Mode Control - General
** Note: Tuning is beyond the scope of this FRS. Understanding of a PID controller is required to understand PID requirements.
## Users will be able to select a Set Point (SP) as the pH target
## A pair of standard PID controllers will be used to seek SP based on the current PV. One for CO2, the other for base. 
# Auto Mode Control - CO2
## When PV is less than SP plus System Variable "Deadband", output will be forced to 0. 
## The PID will use the following System Variables for standard PID input parameters:
### P-Gain: "CO2 P Gain (%/pH)
### I-Time: "CO2 I Time (min)
### D-Time: "CO2 D Time (min)
### α: "CO2 Alpha"
### β: "CO2 Beta"
### γ: "CO2 Gamma"
### Linearity: "CO2 Linearity"
## Output will not be greater than System Variable "CO2 Auto Max (%)"
# Auto Mode Control - Base
## When PV is greater than SP minus System Variable "Deadband", output will be forced to 0. 
## The PID will use the following System Variables for standard PID input parameters:
### P-Gain: "Base P Gain (%/pH)
### I-Time: "Base I Time (min)
### D-Time: "Base D Time (min)
### α: "Base Alpha"
### β: "Base Beta"
### γ: "Base Gamma"
### Linearity: "Base Linearity"
## Output will not be greater than System Variable "Base Auto Max (%)"
# Manual Mode Control
## Users will be able to select outputs for CO2 flow as percentage of main gas flow and base as percentage of max duty cycle. 
## The system will output the user's request when not limited by interlocks, settings, or other specified circumstances. 
## CO2 output will not be greater than System Variable "CO2 Manual Max (%)"
## Base output will not be greater than System Variable "Base Manual Max (%)"

"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)


*+URS3653.1+* The software will be able to control the pH

*+FRS3653+* pH Control
* Note: All system variables in the following items will be located under the category "Temperature" unless specified otherwise.
*+FRS3653.1+* The pH controller will have 2 outputs: % CO2 composition in Main Gas, and % Base Pump Duty.
-*+FRS3653.2+*- Auto mode - software takes the user set point plus the pH "Deadband" setting, and varies the CO2 output to achieve it, based on the present value reported by the sensor and the CO2 PID settings. It also takes the user set point minus the pH "Deadband" setting, and varies the Base output to achieve it, based on the present value reported by the sensor and the Base PID settings.
-*+FRS3653.3+*- Manual Mode - CO2 and Base output are what the user requested.
*+FRS3653.4+* Off mode - CO2 and Base output are 0.
-*+FRS3653.5+*- Broken Sensor mode - when the user has requested Auto mode but the pH PV reported by the sensor is either above the pH "Valid High" setting 